# Efficiency Analysis 


## Objective and Prerequisites

How can mathematical optimization be used to measure the efficiency of an organization? Find out in this example, where you’ll learn how to formulate an Efficiency Analysis model as a linear programming problem using the Gurobi Python API and then generate an optimal solution with the Gurobi Optimizer.

This model is example 22 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 278-280 and 335-336.

This example is at the intermediate level, where we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

## Background

 The Data Envelopment Analysis (DEA) is a nonparametric problem in operations research and economics whose solution is an estimation of production frontiers. It is used to empirically measure the productive efficiency of decision making units (DMUs). There are a number of linear programming formulations of the DEA problem. Fuller coverage of the subject can be found in Farrell (1957), Charnes et al. (1978) and Thanassoulis et al. (1987). The formulation given by H.P. Williams is described in Land (1991). This formulation is the dual model of a model  commonly used that relies on finding weighted ratios of outputs to inputs. We will use the formulation that is commonly used and can be found in Cooper et al. (2007).

The Data Envelopment Analysis has been used to evaluate the performance of many different kinds of entities engaged in many different activities, and in many different contexts in many different countries. Examples include the maintenance activities of U.S. Air Force bases in different geographic locations, or police forces in England and Wales as well as the performance of branch banks in Cyprus and Canada and the efficiency of universities in performing their education and research functions in the U.S., England and France. 

The DEA approach is concerned with evaluations of *efficiency*. The most common measure of efficiency takes the form of a ratio like the following one:

$$
\text{efficiency} = \frac{\text{output}}{\text{input}}
$$



## Model Formulation

Assume there is a set of DMUs. Some common input and output items for each of these DMUs are selected as follows:
1. Numerical data are available for each input and output, with the data assumed to be positive, for all DMUs.
2. The items (inputs, outputs and choice of DMUs) should reflect an analyst's or a manager's interest in the components that will enter into the relative efficiency evaluations of the DMUs.
3. In principle, smaller input amounts are preferable and larger output amounts are preferable so the efficiency scores should reflect these principles.
4. The measurement units of the different inputs and outputs do not need to be congruent. Some may involve a number of persons, or areas of floor space, money expended, etc.

### Fractional problem formulation
The proposed measure of the efficiency of a target DMU $k$ is obtained as the maximum of a ratio of weighted outputs to weighted inputs subject to the condition that the similar ratios for every DMU be less than or equal to one.

### Sets and indices

$j,k \in \text{DMUS}$: Indices and set of DMUs, where $k$ represents the target DMU.

$i \in \text{Inputs}$: Index and set of inputs.

$r \in \text{Outputs}$: Index and set of outputs.

### Parameters

$\text{invalue}_{i,j} > 0$: Value of input $i$ for DMU $j$.

$\text{outvalue}_{r,j} > 0$: Value of output $r$ for DMU $j$.

### Decision Variables

$u_{r} \geq 0$: Weight of output $r$.

$v_{i} \geq 0$: Weight of input  $i$.

### Objective function

**Target DMU Efficiency**: Maximize efficiency at the target DMU $k$.

$$
\text{Maximize} \quad E_k = 
\frac{\sum_{r \in \text{Outputs}} \text{outvalue}_{r,k}*u_{r}}{\sum_{i \in \text{Inputs}} \text{invalue}_{i,k}*v_{i}}
\tag{FP0}
$$


### Constraints

**Efficiency ratios**: The efficiency of a DMU is a number between $[0,1]$.

\begin{equation}
\frac{\sum_{r \in \text{Outputs}} \text{outvalue}_{r,j}*u_{r}}{\sum_{i \in \text{Inputs}} \text{invalue}_{i,j}*v_{i}}
 \leq 1 \quad \forall j \in \text{DMUS}
 \tag{FP1}
\end{equation}



### Linear programming problem formulation

This linear programming formulation can be found in the book by Cooper et al. (2007).

### Objective function

**Target DMU Efficiency**: Maximize efficiency at the target DMU $k$.

$$
\text{Maximize} \quad E_k = \sum_{r \in \text{Outputs}} \text{outvalue}_{r,k}*u_{r}
\tag{LP0}
$$


### Constraints

**Efficiency ratio**: The efficiency of a DMU is a number between $[0,1]$.

\begin{equation}
\sum_{r \in \text{Outputs}} \text{outvalue}_{r,j}*u_{r} -
\sum_{i \in \text{Inputs}} \text{invalue}_{i,k}*v_{i}
 \leq 0  \quad \forall j \in \text{DMUS}
\tag{LP1}
\end{equation}

**Normalization**: This constraint ensures that the denominator of the objective function of the fractional problem is equal to one.

\begin{equation}
\sum_{i \in \text{Inputs}} \text{invalue}_{i,k}*v_{i} = 1 
\tag{LP2}
\end{equation}

It is easy to verify that the fractional problem and the linear programming problem are equivalent. Let's assume that the denominator of the efficiency ratio constraints of the fractional problem is positive for all DMUs, then we can obtain the constraints $LP1$ by multiplying both sides of the constraints $FP1$ by the denominator. Next, we set the denominator of $FP0$ eqaul to 1 and define constraint $LP2$, and then maximize the numerator, resulting in the objective function $LP0$.

### Definition of efficiency

1. $DMU_k$ is efficient if the optimal objective function value $E_{k}^{*} = 1$.
2. Otherwise, $DMU_k$ is inefficient.

## Problem Description

A car manufacturer wants to evaluate the efficiencies of different garages that have been granted a license to sell its cars. Each garage has a certain number of measurable ‘inputs’:
* Staff 
* Showroom Space
* Population in category 1
* Population in category 2
* Enquiries Alpha model
* Enquiries Beta model

Each garage also has a certain number of measurable ‘outputs’:
* Number Sold of different brands of car 
* annual Profit

The following table gives the inputs and outputs for each of the 28 franchised garages. 

![inputOutput1](https://github.com/Gurobi/modeling-examples/blob/master/efficiency_analysis/inputOutput1.PNG?raw=1)
![inputOutput2](https://github.com/Gurobi/modeling-examples/blob/master/efficiency_analysis/inputOutput2.PNG?raw=1)

The goal is to identify efficient and inefficient garages and their input-output weights. In order to solve this problem, it is necessary to solve the LP model for each garage.

---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

### Helper Functions

* `solve_DEA` builds and solves the LP model.

In [4]:
%pip install gurobipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

In [2]:
def solve_DEA(target, verbose=True):
    # input-output values for the garages
    inattr = ['staff', 'showRoom', 'Population1', 'Population2', 'alphaEnquiries', 'betaEnquiries']
    outattr = ['alphaSales', 'BetaSales', 'profit']
    
    dmus, inputs, outputs = gp.multidict({
        'Winchester': [{'staff': 7, 'showRoom': 8, 'Population1': 10, 'Population2': 12, 'alphaEnquiries': 8.5, 'betaEnquiries': 4}, {'alphaSales': 2, 'BetaSales': 0.6, 'profit': 1.5}],
        'Andover': [{'staff': 6, 'showRoom': 6, 'Population1': 20, 'Population2': 30, 'alphaEnquiries': 9, 'betaEnquiries': 4.5}, {'alphaSales': 2.3, 'BetaSales': 0.7, 'profit': 1.6}],
        'Basingstoke': [{'staff': 2, 'showRoom': 3, 'Population1': 40, 'Population2': 40, 'alphaEnquiries': 2, 'betaEnquiries': 1.5}, {'alphaSales': 0.8, 'BetaSales': 0.25, 'profit': 0.5}],
        'Poole': [{'staff': 14, 'showRoom': 9, 'Population1': 20, 'Population2': 25, 'alphaEnquiries': 10, 'betaEnquiries': 6}, {'alphaSales': 2.6, 'BetaSales': 0.86, 'profit': 1.9}],
        'Woking': [{'staff': 10, 'showRoom': 9, 'Population1': 10, 'Population2': 10, 'alphaEnquiries': 11, 'betaEnquiries': 5}, {'alphaSales': 2.4, 'BetaSales': 1, 'profit': 2}],
        'Newbury': [{'staff': 24, 'showRoom': 15, 'Population1': 15, 'Population2': 13, 'alphaEnquiries': 25, 'betaEnquiries': 1.9}, {'alphaSales': 8, 'BetaSales': 2.6, 'profit': 4.5}],
        'Portsmouth': [{'staff': 6, 'showRoom': 7, 'Population1': 50, 'Population2': 40, 'alphaEnquiries': 8.5, 'betaEnquiries': 3}, {'alphaSales': 2.5, 'BetaSales': 0.9, 'profit': 1.6}]
    })
    
    ### Create LP model
    model = gp.Model('DEA') # create model object
    
    # Decision variables
    wout = model.addVars(outattr, name="outputWeight")  # output weights (w)
    win = model.addVars(inattr, name="inputWeight") # input weights (v)

    # Constraints
    ratios = model.addConstrs( ( gp.quicksum(outputs[h][r]*wout[r] for r in outattr ) 
                                - gp.quicksum(inputs[h][i]*win[i] for i in inattr ) 
                                <= 0 for h in dmus ), name='ratios' ) # output ratio constraints
    
    normalization = model.addConstr((gp.quicksum(inputs[target][i]*win[i] for i in inattr ) == 1 ),
                                    name='normalization') # sum of mutliplicative weights and target DMU inputs = 1

    # add input and output weights must be greater and equal to 0
    model.addConstrs((wout[r] >= 0 for r in outattr), name='wout') 

    model.addConstrs((win[i] >= 0 for i in inattr), name='win')
    
    # Objective function (maximize profit)
    model.setObjective( gp.quicksum(outputs[target][r]*wout[r] for r in outattr ), GRB.MAXIMIZE)
    
    # Run optimization engine
    if not verbose:
        model.params.OutputFlag = 0
    model.optimize()
    
    # Print results
    print(f"\nThe efficiency of target DMU {target} is {round(model.objVal,3)}") 
    
    print("__________________________________________________________________")
    print(f"The weights for the inputs are:")
    for i in inattr:
        print(f"For {i}: {round(win[i].x,3)} ") 
        
    print("__________________________________________________________________")
    print(f"The weights for the outputs are")
    for r in outattr:
        print(f"For {r} is: {round(wout[r].x,3)} ") 
    print("__________________________________________________________________\n\n")  
    
    return model.objVal

## Input Data
We define the list of garages.

In [3]:
dmus = ['Winchester','Andover','Basingstoke', 'Poole', 'Woking','Newbury','Portsmouth']

---
## Output Report

We print out the efficiency score of each garage and its associated input and output weights.

In [4]:
# Solving DEA model for each DMU

performance = {}
for h in dmus:    
    performance[h] = solve_DEA(h, verbose=False)


Restricted license - for non-production use only - expires 2024-10-28

The efficiency of target DMU Winchester is 1.0
__________________________________________________________________
The weights for the inputs are:
For staff: 0.12 
For showRoom: 0.0 
For Population1: 0.0 
For Population2: 0.013 
For alphaEnquiries: 0.0 
For betaEnquiries: 0.0 
__________________________________________________________________
The weights for the outputs are
For alphaSales is: 0.0 
For BetaSales is: 0.0 
For profit is: 0.667 
__________________________________________________________________



The efficiency of target DMU Andover is 1.0
__________________________________________________________________
The weights for the inputs are:
For staff: 0.167 
For showRoom: 0.0 
For Population1: 0.0 
For Population2: 0.0 
For alphaEnquiries: 0.0 
For betaEnquiries: 0.0 
__________________________________________________________________
The weights for the outputs are
For alphaSales is: 0.0 
For BetaSales is: 

In [6]:
performance.items()

dict_items([('Winchester', 1.0), ('Andover', 1.0), ('Basingstoke', 1.0), ('Poole', 0.9999999999999999), ('Woking', 1.0000000000000002), ('Newbury', 1.0), ('Portsmouth', 1.0)])

---
## Analysis

We identify which garages are efficient and which ones are inefficient, and provide the efficiency scores for each garage.

In [ ]:
# Identifying efficient and inefficient DMUs

# Sorting garages in descending efficiency number
sorted_performance = {k: v for k, v in sorted(performance.items(), key=lambda item: item[1], reverse = True)}

efficient = []
inefficient = []

for h in sorted_performance.keys():
    if sorted_performance[h] >= 0.9999999:
        efficient.append(h) 
    if sorted_performance[h] < 0.9999999:
        inefficient.append(h)
        
print('____________________________________________')
print(f"The efficient DMUs are:")
for eff in efficient:
    print(f"The performance value of DMU {eff} is: {round(performance[eff],3)}") 
    
print('____________________________________________')
print(f"The inefficient DMUs are:")
for ine in inefficient:
    print(f"The performance value of DMU {ine} is: {round(performance[ine],3)}") 


____________________________________________
The efficient DMUs are:
The performance value of DMU Newbury is: 1.0
The performance value of DMU Alresford is: 1.0
The performance value of DMU Salisbury is: 1.0
The performance value of DMU Alton is: 1.0
The performance value of DMU Weymouth is: 1.0
The performance value of DMU Petersfield is: 1.0
The performance value of DMU Southampton is: 1.0
The performance value of DMU Bournemouth is: 1.0
The performance value of DMU Maidenhead is: 1.0
The performance value of DMU Fareham is: 1.0
The performance value of DMU Romsey is: 1.0
The performance value of DMU Basingstoke is: 1.0
The performance value of DMU Portsmouth is: 1.0
The performance value of DMU Portland is: 1.0
The performance value of DMU Henley is: 1.0
____________________________________________
The inefficient DMUs are:
The performance value of DMU Petworth is: 0.988
The performance value of DMU Reading is: 0.984
The performance value of DMU Bridport is: 0.982
The performance va

## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Cooper, W. W, L. M. Seiford, K. Tone. (2007) Data Envelopment Analysis: A Comprehensive Text with Models, Applications, References and DEA-Solver Software. Second edition. Springer-Verlag US.

Land, A. (1991) Data envelopment analysis, Chapter 5, in Operations Research in Management (eds S.C. Littlechild and M.F. Shutler), Prentice Hall, London.

Farrell, M.J. (1957) The measurement of productive efficiency. Journal of the Royal Statistical Society, Series A, 120, 253–290.

Charnes, A., Cooper, W.W. and Rhodes, E. (1978) Measuring the efficiency of decision making units. European Journal of Operational Research, 2, 429–444.

Thanassoulis, E., Dyson, R.G. and Foster, M.J. (1987) Relative efficiency assessments using data envelopment analysis: an application to data on rates departments. Journal of the Operational Research Society, 5, 397–411.

Copyright © 2020 Gurobi Optimization, LLC

In [1]:
import gurobipy as gp

dmus, inputs, outputs = gp.multidict({
    'Line_1': [{'Electricity': 1, 'Gas': 8, 'Water': 10, 'Chemical': 12, 'Maintance': 8.5, 'Labor': 4},   {'Production_Amount': 2,    'Quality': 0.6}],
    'Line_2': [{'Electricity': 1, 'Gas': 6, 'Water': 20, 'Chemical': 30, 'Maintance': 9,   'Labor': 4.5}, {'Production_Amount': 2.3,  'Quality': 0.7}],
    'Line_3': [{'Electricity': 2, 'Gas': 3, 'Water': 40, 'Chemical': 40, 'Maintance': 2,   'Labor': 1.5}, {'Production_Amount': 0.8,  'Quality': 0.25}],
    'Line_4': [{'Electricity': 2, 'Gas': 9, 'Water': 20, 'Chemical': 25, 'Maintance': 10,  'Labor': 6},   {'Production_Amount': 2.6,  'Quality': 0.86}]
})


In [2]:
dmus

['Line_1', 'Line_2', 'Line_3', 'Line_4']

In [3]:
inputs

{'Line_1': {'Electricity': 1,
  'Gas': 8,
  'Water': 10,
  'Chemical': 12,
  'Maintance': 8.5,
  'Labor': 4},
 'Line_2': {'Electricity': 1,
  'Gas': 6,
  'Water': 20,
  'Chemical': 30,
  'Maintance': 9,
  'Labor': 4.5},
 'Line_3': {'Electricity': 2,
  'Gas': 3,
  'Water': 40,
  'Chemical': 40,
  'Maintance': 2,
  'Labor': 1.5},
 'Line_4': {'Electricity': 2,
  'Gas': 9,
  'Water': 20,
  'Chemical': 25,
  'Maintance': 10,
  'Labor': 6}}

In [8]:
import pandas as pd
inputss_=pd.DataFrame(inputs).T
inputss_

,Electricity,Gas,Water,Chemical,Maintance,Labor
Line_1,1.0,8.0,10.0,12.0,8.5,4.0
Line_2,1.0,6.0,20.0,30.0,9.0,4.5
Line_3,2.0,3.0,40.0,40.0,2.0,1.5
Line_4,2.0,9.0,20.0,25.0,10.0,6.0


In [4]:
outputs

{'Line_1': {'Production_Amount': 2, 'Quality': 0.6},
 'Line_2': {'Production_Amount': 2.3, 'Quality': 0.7},
 'Line_3': {'Production_Amount': 0.8, 'Quality': 0.25},
 'Line_4': {'Production_Amount': 2.6, 'Quality': 0.86}}